## Linear Classifier in TensorFlow 
Using Low Level API in Eager Execution mode

### Load tensorflow

In [1]:
%tensorflow_version 2.x
import tensorflow as tf
import numpy as np

TensorFlow 2.x selected.


In [2]:
print(tf.__version__)

2.0.0


In [0]:
#Enable Eager Execution if using tensflow version < 2.0
#From tensorflow v2.0 onwards, Eager Execution will be enabled by default
#tf.enable_eager_execution()

### Collect Data

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import pandas as pd

In [0]:
data = pd.read_csv('/content/gdrive/My Drive/prices.csv')

### Check all columns in the dataset

In [6]:
data.head()

,date,symbol,open,close,low,high,volume
0,2016-01-05 00:00:00,WLTW,123.430000,125.839996,122.309998,126.250000,2163600.0
1,2016-01-06 00:00:00,WLTW,125.239998,119.980003,119.940002,125.540001,2386400.0
2,2016-01-07 00:00:00,WLTW,116.379997,114.949997,114.930000,119.739998,2489500.0
3,2016-01-08 00:00:00,WLTW,115.480003,116.620003,113.500000,117.440002,2006300.0
4,2016-01-11 00:00:00,WLTW,117.010002,114.970001,114.089996,117.330002,1408600.0


In [7]:
data.columns

Index(['date', 'symbol', 'open', 'close', 'low', 'high', 'volume'], dtype='object')

### Drop columns `date` and  `symbol`

In [0]:
data = data.drop(['date','symbol'], axis = 1)

In [9]:
data.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2163600.0
1,125.239998,119.980003,119.940002,125.540001,2386400.0
2,116.379997,114.949997,114.930000,119.739998,2489500.0
3,115.480003,116.620003,113.500000,117.440002,2006300.0
4,117.010002,114.970001,114.089996,117.330002,1408600.0


### Consider only first 1000 rows in the dataset for building feature set and target set
Target 'Volume' has very high values. Divide 'Volume' by 1000,000

In [10]:
data.shape

(851264, 5)

In [11]:
data = data.iloc[:1000,:]
data.shape

(1000, 5)

In [12]:
data['volume'] = data['volume']/1000000
data['volume'].describe()

count    1000.000000
mean        5.314222
std        14.465650
min         0.010000
25%         0.821625
50%         2.064450
75%         4.620475
max       215.620200
Name: volume, dtype: float64

In [0]:
X = data.drop('volume', axis = 1)
Y = data['volume']

### Divide the data into train and test sets

In [0]:
from sklearn.model_selection import train_test_split
train_x,test_x,train_y,test_y=train_test_split(X,Y,test_size=.30,random_state=2)

#### Convert Training and Test Data to numpy float32 arrays


In [15]:
np.array(train_x).dtype

dtype('float64')

In [0]:
train_x =np.array(train_x).astype('float32')
test_x = np.array(test_x).astype('float32')
train_y =np.array(train_y).astype('float32')
test_y = np.array(test_y).astype('float32')

In [17]:
np.array(train_x).dtype

dtype('float32')

### Normalize the data
You can use Normalizer from sklearn.preprocessing

In [0]:
from sklearn.preprocessing import Normalizer

transformer = Normalizer() #same as Z-score
train_x = transformer.fit_transform(train_x)
test_x = transformer.transform(test_x)

In [19]:
train_y.shape

(700,)

In [0]:
train_y=train_y.reshape(-1,1)

In [21]:
train_y.shape

(700, 1)

In [22]:
train_x.shape

(700, 4)

## Building the Model in tensorflow

1.Define Weights and Bias, use tf.zeros to initialize weights and Bias

In [0]:
#We are initializing weights and Bias of (Input-Hidden Layer)
w1 = tf.random.normal(shape=(4,4))
b1 = tf.zeros(shape=(4))

In [0]:
#We are initializing weights and Bias of (Hidden-Output)
w2 = tf.random.normal(shape=(4,1))
b2 = tf.zeros(shape=(1))

2.Define a function to calculate prediction

In [0]:
def prediction(x, w1, b1,w2,b2):
    
    xw_matmul = tf.matmul(x, w1)
    net1 = tf.add(xw_matmul, b1)
    y=tf.sigmoid(net1)
    net2=tf.matmul(y, w2)+b2
    out=tf.sigmoid(net2)
    return net2,out

## net2 for regression and out for classification

3.Loss (Cost) Function [Mean square error]

In [0]:
def loss(predicted_y, desired_y):
  return tf.reduce_mean(tf.square(predicted_y - desired_y))

4.Function to train the Model

1.   Record all the mathematical steps to calculate Loss
2.   Calculate Gradients of Loss w.r.t weights and bias
3.   Update Weights and Bias based on gradients and learning rate to minimize loss

In [0]:
def train(train_x, train_y, w1, b1,w2,b2, learning_rate=0.01):
    
    #Record mathematical operations on 'tape' to calculate loss
    with tf.GradientTape() as t:
        
        t.watch([w1,b1,w2,b2]) #which values have to be updated/keep track of/model parameters
        
        net2,current_prediction = prediction(train_x, w1, b1,w2,b2)
        current_loss =loss(net2, train_y)
    
    #Calculate Gradients for Loss with respect to Weights and Bias
    dw1,db1,dw2,db2 = t.gradient(current_loss,[w1, b1,w2,b2])
    
    #Update Weights at output layer
    w2 = w2 - learning_rate*dw2
    b2 = b2 - learning_rate*db2
    
     #Update Weights at hidden layer
    w1 = w1 - learning_rate*dw1
    b1 = b1 - learning_rate*db1
    
    return w1, b1,w2,b2, current_loss

## Train the model for 100 epochs 
1. Observe the training loss at every iteration
2. Observe Train loss at every 5th iteration

In [29]:
for i in range(100):
    
    w1, b1,w2,b2, current_loss = train(train_x, train_y, w1, b1,w2,b2)
    print("Loss at step {:d}: {:.3f}".format(i, current_loss))

Loss at step 0: 252.931
Loss at step 1: 250.618
Loss at step 2: 248.476
Loss at step 3: 246.487
Loss at step 4: 244.634
Loss at step 5: 242.904
Loss at step 6: 241.287
Loss at step 7: 239.773
Loss at step 8: 238.354
Loss at step 9: 237.022
Loss at step 10: 235.773
Loss at step 11: 234.599
Loss at step 12: 233.498
Loss at step 13: 232.463
Loss at step 14: 231.493
Loss at step 15: 230.582
Loss at step 16: 229.728
Loss at step 17: 228.928
Loss at step 18: 228.178
Loss at step 19: 227.477
Loss at step 20: 226.822
Loss at step 21: 226.209
Loss at step 22: 225.638
Loss at step 23: 225.105
Loss at step 24: 224.608
Loss at step 25: 224.146
Loss at step 26: 223.717
Loss at step 27: 223.318
Loss at step 28: 222.947
Loss at step 29: 222.604
Loss at step 30: 222.285
Loss at step 31: 221.991
Loss at step 32: 221.718
Loss at step 33: 221.466
Loss at step 34: 221.233
Loss at step 35: 221.019
Loss at step 36: 220.821
Loss at step 37: 220.638
Loss at step 38: 220.470
Loss at step 39: 220.316
Loss at st

In [30]:
j = 0
for i in range(100):
    j = j+1
    w1, b1,w2,b2, current_loss = train(train_x, train_y, w1, b1,w2,b2)
    if(j ==5):
      print("Loss at step {:d}: {:.3f}".format(i, current_loss))
      j = 0

Loss at step 4: 218.650
Loss at step 9: 218.648
Loss at step 14: 218.647
Loss at step 19: 218.647
Loss at step 24: 218.647
Loss at step 29: 218.646
Loss at step 34: 218.646
Loss at step 39: 218.646
Loss at step 44: 218.646
Loss at step 49: 218.646
Loss at step 54: 218.646
Loss at step 59: 218.646
Loss at step 64: 218.646
Loss at step 69: 218.646
Loss at step 74: 218.646
Loss at step 79: 218.646
Loss at step 84: 218.646
Loss at step 89: 218.646
Loss at step 94: 218.646
Loss at step 99: 218.646


### Get the shapes and values of W and b

In [31]:
w1.shape

TensorShape([4, 4])

In [32]:
w2.shape

TensorShape([4, 1])

In [33]:
b1.shape

TensorShape([4])

In [34]:
b2.shape

TensorShape([1])

### Model Prediction on 1st Examples in Test Dataset

In [35]:
y_pred,_=prediction(test_x,w1,b1,w2,b2)
y_pred

<tf.Tensor: id=10824, shape=(300, 1), dtype=float32, numpy=
array([[5.6540956],
       [5.6491766],
       [5.651331 ],
       [5.648365 ],
       [5.6510344],
       [5.6540847],
       [5.649946 ],
       [5.6476574],
       [5.651206 ],
       [5.6550436],
       [5.650572 ],
       [5.647544 ],
       [5.6478214],
       [5.65282  ],
       [5.64717  ],
       [5.6489334],
       [5.6600137],
       [5.650756 ],
       [5.6477213],
       [5.647435 ],
       [5.6512322],
       [5.6477056],
       [5.6518526],
       [5.6518993],
       [5.6493607],
       [5.6501083],
       [5.6489296],
       [5.651355 ],
       [5.6480236],
       [5.6477046],
       [5.6574345],
       [5.651604 ],
       [5.649398 ],
       [5.649975 ],
       [5.6476345],
       [5.649069 ],
       [5.6488395],
       [5.647338 ],
       [5.6503124],
       [5.6502256],
       [5.6548834],
       [5.647975 ],
       [5.648081 ],
       [5.650653 ],
       [5.649844 ],
       [5.6530833],
       [5.6504126],


In [43]:
y_pred,_=prediction(test_x[:1,:],w1,b1,w2,b2)
print("The predicted value for the first row in test set is = ",y_pred.numpy())

The predicted value for the first row in test set is =  [[5.654095]]


In [45]:
from sklearn import metrics
y_pred,_=prediction(test_x,w1,b1,w2,b2)
cm=metrics.r2_score(test_y,y_pred)
print('The r2 value for the predicted data set is',cm)

The r2 value for the predicted data set is -0.006692816626929998


In [46]:
print('The loss is',loss(y_pred, test_y).numpy())

The loss is 187.02774


## Classification using tf.Keras

In this exercise, we will build a Deep Neural Network using tf.Keras. We will use Iris Dataset for this exercise.

### Load the given Iris data using pandas (Iris.csv)

In [0]:
data_iris = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/Iris.csv')

### Target set has different categories. So, Label encode them. And convert into one-hot vectors using get_dummies in pandas.

In [48]:
data_iris.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


### Splitting the data into feature set and target set

In [0]:
X = data_iris.drop('Species', axis = 1)
Y = data_iris['Species']

In [50]:
X.shape

(150, 5)

In [51]:
X = X.drop('Id', axis = 1)
X.shape

(150, 4)

In [52]:
Y.shape

(150,)

In [53]:
Y=np.array(Y).reshape(-1,1)
Y = pd.DataFrame(Y)
Y.shape

(150, 1)

In [0]:
Y['species'] = Y

In [0]:
Y = Y.drop(0, axis = 1)

In [56]:
Y.head()

,species
0,Iris-setosa
1,Iris-setosa
2,Iris-setosa
3,Iris-setosa
4,Iris-setosa


In [57]:
Y =pd.get_dummies(Y)
Y.head()

,species_Iris-setosa,species_Iris-versicolor,species_Iris-virginica
0,1,0,0
1,1,0,0
2,1,0,0
3,1,0,0
4,1,0,0


In [0]:
train_x,test_x,train_y,test_y=train_test_split(X,Y,test_size=.30,random_state=2)

In [0]:
train_x =np.array(train_x).astype('float32')
test_x = np.array(test_x).astype('float32')
train_y =np.array(train_y).astype('float32')
test_y = np.array(test_y).astype('float32')

###  Building Model in tf.keras

Build a Linear Classifier model  <br>
1.  Use Dense Layer  with input shape of 4 (according to the feature set) and number of outputs set to 3<br> 
2. Apply Softmax on Dense Layer outputs <br>
3. Use SGD as Optimizer
4. Use categorical_crossentropy as loss function 

In [60]:
from keras.models import Sequential
from keras.layers import Dense


Using TensorFlow backend.


In [0]:
#tf.compat.v1.disable_eager_execution()
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(4, input_dim = 4, kernel_initializer='normal', activation='softmax'))
model.add(tf.keras.layers.Dense(3, activation='softmax'))    
    # Compile model
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics = ['accuracy'])


### Model Training 

In [65]:

model.fit(train_x, train_y,validation_data = (test_x,test_y), epochs = 50)
#print(output)

Train on 105 samples, validate on 45 samples
Epoch 1/50
105/105 [==============================] - 1s 9ms/sample - loss: 1.0724 - accuracy: 0.3524 - val_loss: 1.0799 - val_accuracy: 0.2889
Epoch 2/50
105/105 [==============================] - 0s 254us/sample - loss: 1.0680 - accuracy: 0.3524 - val_loss: 1.0765 - val_accuracy: 0.2889
Epoch 3/50
105/105 [==============================] - 0s 252us/sample - loss: 1.0649 - accuracy: 0.3524 - val_loss: 1.0730 - val_accuracy: 0.2889
Epoch 4/50
105/105 [==============================] - 0s 250us/sample - loss: 1.0588 - accuracy: 0.3524 - val_loss: 1.0687 - val_accuracy: 0.2889
Epoch 5/50
105/105 [==============================] - 0s 266us/sample - loss: 1.0543 - accuracy: 0.3524 - val_loss: 1.0642 - val_accuracy: 0.2889
Epoch 6/50
105/105 [==============================] - 0s 230us/sample - loss: 1.0494 - accuracy: 0.3524 - val_loss: 1.0596 - val_accuracy: 0.2889
Epoch 7/50
105/105 [==============================] - 0s 259us/sample - loss: 1.0

### Model Prediction

In [66]:

y_pred = model.predict(test_x)
y_pred

array([[0.54598814, 0.23632866, 0.21768321],
       [0.52592987, 0.24454494, 0.22952521],
       [0.19251   , 0.3256129 , 0.48187703],
       [0.5368166 , 0.2402098 , 0.2229736 ],
       [0.5164635 , 0.2475341 , 0.23600243],
       [0.19266117, 0.32361534, 0.4837235 ],
       [0.523063  , 0.24544026, 0.23149674],
       [0.17915003, 0.3243096 , 0.4965404 ],
       [0.18343964, 0.32450336, 0.49205703],
       [0.52967566, 0.24314384, 0.22718047],
       [0.57353973, 0.2243025 , 0.20215783],
       [0.54100764, 0.2386753 , 0.22031704],
       [0.5625673 , 0.22955817, 0.20787455],
       [0.56584513, 0.22725664, 0.20689818],
       [0.2525164 , 0.3196648 , 0.42781878],
       [0.23399262, 0.3221167 , 0.44389063],
       [0.48887312, 0.26011696, 0.25100997],
       [0.25631937, 0.3201263 , 0.42355433],
       [0.18549272, 0.3239898 , 0.49051747],
       [0.250311  , 0.321896  , 0.42779303],
       [0.21754017, 0.32309604, 0.45936376],
       [0.26061445, 0.31871766, 0.42066792],
       [0.

In [0]:

y_test_class = np.argmax(test_y,axis = 1)
y_pred_class = np.argmax(y_pred, axis = 1)

The location of the maximum probability is checked against the original data

In [68]:
y_pred_class

array([0, 0, 2, 0, 0, 2, 0, 2, 2, 0, 0, 0, 0, 0, 2, 2, 0, 2, 2, 2, 2, 2,
       2, 2, 2, 0, 0, 2, 0, 2, 2, 0, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2,
       0])

In [69]:
from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(y_test_class,y_pred_class))


              precision    recall  f1-score   support

           0       1.00      1.00      1.00        17
           1       0.00      0.00      0.00        15
           2       0.46      1.00      0.63        13

    accuracy                           0.67        45
   macro avg       0.49      0.67      0.54        45
weighted avg       0.51      0.67      0.56        45



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [70]:
print(confusion_matrix(y_test_class,y_pred_class))

[[17  0  0]
 [ 0  0 15]
 [ 0  0 13]]


### Save the Model

In [0]:
model.save('iris_model.h5')

### Build and Train a Deep Neural network with 2 hidden layer  - Optional - For Practice

Does it perform better than Linear Classifier? What could be the reason for difference in performance?

In [0]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(4, input_dim = 4, kernel_initializer='normal', activation='softmax'))
model.add(tf.keras.layers.Dense(3, kernel_initializer='normal', activation='sigmoid'))
model.add(tf.keras.layers.Dense(3, activation='softmax'))    
    # Compile model
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics = ['accuracy'])

In [79]:
model.fit(train_x, train_y,validation_data = (test_x,test_y), epochs = 50)


Train on 105 samples, validate on 45 samples
Epoch 1/50
105/105 [==============================] - 1s 5ms/sample - loss: 1.1497 - accuracy: 0.3143 - val_loss: 1.1096 - val_accuracy: 0.3778
Epoch 2/50
105/105 [==============================] - 0s 278us/sample - loss: 1.1469 - accuracy: 0.3143 - val_loss: 1.1079 - val_accuracy: 0.3778
Epoch 3/50
105/105 [==============================] - 0s 280us/sample - loss: 1.1439 - accuracy: 0.3143 - val_loss: 1.1066 - val_accuracy: 0.3778
Epoch 4/50
105/105 [==============================] - 0s 249us/sample - loss: 1.1415 - accuracy: 0.3143 - val_loss: 1.1056 - val_accuracy: 0.3778
Epoch 5/50
105/105 [==============================] - 0s 283us/sample - loss: 1.1394 - accuracy: 0.3143 - val_loss: 1.1041 - val_accuracy: 0.3778
Epoch 6/50
105/105 [==============================] - 0s 261us/sample - loss: 1.1366 - accuracy: 0.3143 - val_loss: 1.1028 - val_accuracy: 0.3778
Epoch 7/50
105/105 [==============================] - 0s 230us/sample - loss: 1.1

In [80]:

y_pred = model.predict(test_x)
y_pred

array([[0.35388234, 0.31696913, 0.32914862],
       [0.35387444, 0.31696638, 0.32915914],
       [0.35367927, 0.31698474, 0.329336  ],
       [0.3538467 , 0.3169654 , 0.32918796],
       [0.35387638, 0.31697908, 0.32914454],
       [0.3535581 , 0.31701615, 0.32942572],
       [0.35382038, 0.31696776, 0.32921183],
       [0.35358065, 0.3170028 , 0.32941654],
       [0.3536128 , 0.31699997, 0.3293872 ],
       [0.3538347 , 0.3169623 , 0.32920307],
       [0.3538192 , 0.3169809 , 0.32919994],
       [0.35390022, 0.31696436, 0.32913545],
       [0.3538208 , 0.3169655 , 0.32921368],
       [0.353811  , 0.31698507, 0.32920396],
       [0.35370287, 0.31700107, 0.32929605],
       [0.3536135 , 0.3169949 , 0.3293916 ],
       [0.35383326, 0.31694582, 0.3292209 ],
       [0.35372445, 0.31699088, 0.3292847 ],
       [0.3535797 , 0.3170056 , 0.32941467],
       [0.35372338, 0.3169797 , 0.32929686],
       [0.35359636, 0.317002  , 0.32940164],
       [0.35363713, 0.316995  , 0.3293679 ],
       [0.

In [81]:
y_pred_class

array([0, 0, 2, 0, 0, 2, 0, 2, 2, 0, 0, 0, 0, 0, 2, 2, 0, 2, 2, 2, 2, 2,
       2, 2, 2, 0, 0, 2, 0, 2, 2, 0, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2,
       0])

In [82]:
print(classification_report(y_test_class,y_pred_class))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        17
           1       0.00      0.00      0.00        15
           2       0.46      1.00      0.63        13

    accuracy                           0.67        45
   macro avg       0.49      0.67      0.54        45
weighted avg       0.51      0.67      0.56        45



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [83]:
print(confusion_matrix(y_test_class,y_pred_class))

[[17  0  0]
 [ 0  0 15]
 [ 0  0 13]]


There is not much change after adding a layer over the linear model